In [2]:
from tkinter.tix import tixCommand

from torchcrepe.convert import dither

from optispeech.model import OptiSpeech
import torchvision

In [3]:

m: OptiSpeech = OptiSpeech.load_from_checkpoint("../../mike-checkpoint_epoch-729_step-305000.ckpt", map_location="cpu")

/home/hadware/Code/ogmios-workbench/leanspeech/venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Low pass filter created, time used = 0.0007 seconds
num_octave =  9
No early downsampling is required, downsample_factor =  1
Early downsampling filter created,                         time used = 0.0000 seconds
CQT kernels created, time used = 0.0010 seconds
Low pass filter created, time used = 0.0003 seconds
num_octave =  9
No early downsampling is required, downsample_factor =  1
Early downsampling filter created,                         time used = 0.0000 seconds
CQT kernels created, time used = 0.0022 seconds
Low pass filter created, time used = 0.0002 seconds
num_octave =  9
No early downsampling is required, downsample_factor =  1
Early downsampling filter created,                         time used = 0.0000 seconds
CQT kernels created, time used = 0.0031 seconds


In [15]:
def model_size_in_mb(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024 ** 2
    return size_all_mb
model_size_in_mb(m.generator.encoder), model_size_in_mb(m.generator.decoder), model_size_in_mb(m.generator.wav_generator), model_size_in_mb(m.discriminator)

(2.119140625, 2.1352577209472656, 35.365333557128906, 160.89519500732422)

In [5]:
import torch


def module_size_in_mb(module):
    param_size = sum(param.nelement() * param.element_size() for param in module.parameters())
    buffer_size = sum(buffer.nelement() * buffer.element_size() for buffer in module.buffers())
    size_mb = (param_size + buffer_size) / 1024 ** 2
    return size_mb


def print_module_sizes(model, indent=0):
    size_mb = module_size_in_mb(model)
    print(f'{" " * indent}Module: {model.__class__.__name__}, Size: {size_mb:.2f} MB')

    for name, child in model.named_children():
        print_module_sizes(child, indent + 2)


print_module_sizes(m.generator)


Module: OptiSpeechGenerator, Size: 46.26 MB
  Module: TextEmbedding, Size: 0.24 MB
    Module: Embedding, Size: 0.24 MB
    Module: ScaledSinusoidalEmbedding, Size: 0.00 MB
    Module: Dropout, Size: 0.00 MB
  Module: LightSpeechTransformerEncoder, Size: 2.12 MB
    Module: ModuleList, Size: 2.12 MB
      Module: EncSepConvLayer, Size: 0.51 MB
        Module: LayerNorm, Size: 0.00 MB
        Module: ReLU, Size: 0.00 MB
        Module: ConvSeparable, Size: 0.26 MB
          Module: Conv1d, Size: 0.00 MB
          Module: Conv1d, Size: 0.25 MB
        Module: ConvSeparable, Size: 0.26 MB
          Module: Conv1d, Size: 0.00 MB
          Module: Conv1d, Size: 0.25 MB
      Module: EncSepConvLayer, Size: 0.55 MB
        Module: LayerNorm, Size: 0.00 MB
        Module: ReLU, Size: 0.00 MB
        Module: ConvSeparable, Size: 0.28 MB
          Module: Conv1d, Size: 0.02 MB
          Module: Conv1d, Size: 0.25 MB
        Module: ConvSeparable, Size: 0.28 MB
          Module: Conv1d, Size: 0.0

In [12]:
dict(m.generator.named_children())

{'text_embedding': TextEmbedding(
   (embed_tokens): Embedding(250, 256, padding_idx=0)
   (embed_positions): ScaledSinusoidalEmbedding()
   (emb_dropout): Dropout(p=0.1, inplace=False)
 ),
 'encoder': LightSpeechTransformerEncoder(
   (layers): ModuleList(
     (0): EncSepConvLayer(
       (layer_norm): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
       (activation_fn): ReLU(inplace=True)
       (conv1): ConvSeparable(
         (depthwise_conv): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,), groups=256, bias=False)
         (pointwise_conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
       )
       (conv2): ConvSeparable(
         (depthwise_conv): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,), groups=256, bias=False)
         (pointwise_conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
       )
     )
     (1): EncSepConvLayer(
       (layer_norm): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
       (activation_fn): ReLU(inp

In [10]:
from torchview import draw_graph

draw_graph(m.generator, input_size=(64, ))

RuntimeError: Only one of (input_data, input_size) should be specified.